Build a basic chatbot (langraph)

1. Load enviorment

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

Imports and State Definition


In [ ]:
from typing import List, Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq
from pydantic import SecretStr

2. Create State and define variable for the state

In [ ]:
#define state
class State(TypedDict):
    messages: Annotated[List, add_messages]



3. Initialize the LLM Model [groq]

In [ ]:
llm = ChatGroq(
    model="llama3-8b-8192",
    api_key=SecretStr(os.getenv("GROQ_API_KEY"))
)

4. Create chatbot method

In [ ]:
def chatbot(state: State):
    # The LLM expects a list of messages
    return {"messages": [llm.invoke(state["messages"])]}

def message_builder(role, message):
    return {"messages": [{"role": role, "content": message}]}

def invoke_message(graph, message, role, config=None):
    item=message_builder(role, message)
    if not config:
        return graph.invoke(item)
    return graph.invoke(item, config)

5. Build Graph with node and edges [start -> chatbot -> end] 

In [ ]:
graph_builder = StateGraph(State)

#add node
graph_builder.add_node("llm_chatbot", chatbot)

#add edge
graph_builder.add_edge(START, "llm_chatbot")
graph_builder.add_edge("llm_chatbot", END)

#compile graph
graph = graph_builder.compile()

Display the graph

In [ ]:
#visualize graph
from IPython.display import Image, display
def display_graph(graph):
    try:
        display(Image(graph.get_graph().draw_mermaid_png()))
    except Exception as e:
        print(e)
        

In [ ]:
response = invoke_message(graph, "Hello, how are you?", "user")
#graph.invoke({"messages": [{"role": "user", "content": "Hello, how are you?"}]})

In [ ]:
response["messages"][-1].content


In [ ]:
for event in graph.stream({"messages":[{"role": "user", "content": "Hello, how do you you?"}]}):
    for value in event.values():
        print(value["messages"][-1].content)

# Chatbot with Tool

In [ ]:
from langchain_tavily import TavilySearch

tavily_tool=TavilySearch(max_results=2)

In [ ]:
## custom function
def multiply(a:int, b:int) -> int:
    """
        Multiply a and b
        
        Args:
            a:int first int
            b:int second int
        
        Returns:
            int: output 
    """
    return a*b

def pretty_print(response):
    for message in response['messages']:
        message.pretty_print()

In [ ]:
tools=[tavily_tool,multiply]
llm_with_tools = llm.bind_tools(tools)



In [ ]:
## State Graph for tools
from langgraph.prebuilt import ToolNode, tools_condition

## Node defination
def tool_calling_llm(state: State):
      return {"messages": [llm_with_tools.invoke(state["messages"])]}

## Graph
tool_graph_builder = StateGraph(State)
tool_graph_builder.add_node("tool_calling_llm", tool_calling_llm)
tool_graph_builder.add_node("tools",ToolNode(tools))

##add edges
tool_graph_builder.add_edge(START, "tool_calling_llm")


#add conditional edge
tool_graph_builder.add_conditional_edges(
    "tool_calling_llm", 
    # if the latest message (result) from assistant is a tool call -> tools_condition routes it to "tools"
    # if the latest message (result) from assistant is not a tool call -> tools_condition routes to END
    tools_condition
)

tool_graph_builder.add_edge("tools", END)

#compile the graph
graph=tool_graph_builder.compile()
display_graph(graph)

In [ ]:
response= invoke_message(graph, "What is the recent AI news?", "user")
pretty_print(response)

In [ ]:
response=  invoke_message(graph,"what is 2 multiply by 3?" ,"user")
pretty_print(response)


# ReAct agent Architecture

In [ ]:
## State Graph for tools
from langgraph.prebuilt import ToolNode, tools_condition

## Node defination
def tool_calling_llm(state: State):
      return {"messages": [llm_with_tools.invoke(state["messages"])]}

## Graph
tool_graph_builder = StateGraph(State)
tool_graph_builder.add_node("tool_calling_llm", tool_calling_llm)
tool_graph_builder.add_node("tools",ToolNode(tools))

##add edges
tool_graph_builder.add_edge(START, "tool_calling_llm")


#add conditional edge
tool_graph_builder.add_conditional_edges(
    "tool_calling_llm", 
    # if the latest message (result) from assistant is a tool call -> tools_condition routes it to "tools"
    # if the latest message (result) from assistant is not a tool call -> tools_condition routes to END
    tools_condition
)

tool_graph_builder.add_edge("tools", "tool_calling_llm") # Agentic nature added


#compile the graph
graph=tool_graph_builder.compile()
display_graph(graph)

In [ ]:
response= invoke_message(graph,"Please update me with recent AI news and than 2 multiply by 3?" ,"user")
pretty_print(response)


## Adding Memory in Agenting Graph

In [ ]:
## State Graph for tools
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver # memory for llm 

## Node defination
def tool_calling_llm(state: State):
      return {"messages": [llm_with_tools.invoke(state["messages"])]}

## Graph
tool_graph_builder = StateGraph(State)
tool_graph_builder.add_node("tool_calling_llm", tool_calling_llm)
tool_graph_builder.add_node("tools",ToolNode(tools))

##add edges
tool_graph_builder.add_edge(START, "tool_calling_llm")


#add conditional edge
tool_graph_builder.add_conditional_edges(
    "tool_calling_llm", 
    # if the latest message (result) from assistant is a tool call -> tools_condition routes it to "tools"
    # if the latest message (result) from assistant is not a tool call -> tools_condition routes to END
    tools_condition
)

tool_graph_builder.add_edge("tools", "tool_calling_llm") # Agentic nature added


#compile the graph
memory = MemorySaver()
graph=tool_graph_builder.compile(checkpointer=memory)
display_graph(graph)

In [ ]:
def thread_config(id=1):
    return {"configurable": {"thread_id": "1"}}
    

In [ ]:
config = thread_config() # Example, adjust as needed
response = invoke_message(graph,"My name is Nitin" ,"user", config) 
response

In [ ]:
response = invoke_message(graph,"What is My name " ,"user", config)  
response['messages'][-1].content

In [ ]:
response = invoke_message(graph,"Do you remember me  " ,"user", config) 
response['messages'][-1].content

Streaming
Methods: .stream() and astream()

These methods are sync and async methods for streaming back results.
Additional parameters in streaming modes for graph state

values : This streams the full state of the graph after each node is called.
updates : This streams updates to the state of the graph after each node is called.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

def superbot(state:State):
    return {"messages":[llm_with_tools.invoke(state['messages'])]}

In [ ]:
graph_builder = StateGraph(State)

#node
graph_builder.add_node("Superbot", superbot )
graph_builder.add_edge(START, "Superbot")
graph_builder.add_edge("Superbot", END)

graph = graph_builder.compile(checkpointer=memory)
display_graph(graph)


In [ ]:
## invocation
config = thread_config() # Example, adjust as needed

response = invoke_message(graph,"My name is Nitin and i like video games" ,"user", config) 
pretty_print(response)

Stream: Update mode

In [ ]:
config = thread_config(3)

#updates
for chunk in graph.stream(message_builder("user", "My name is Nitin and i like video games"), config, stream_mode="updates"):
    # your code here 
    print(chunk)


In [ ]:
#values
for chunk in graph.stream(message_builder("user", "My name is Nitin and i like video games"), config=config, stream_mode="values"):
    # your code here 
    print(chunk)


# Async mode

In [ ]:
config = thread_config(4)

#updates
async for event in graph.astream_events(message_builder("user", "My name is Nitin and i like video games"), config, version="v2", stream_mode="updates"):
    # your code here 
    print(event)


Human in the loop